# Machine Learning Diabetes Classification

## Read csv and perform basic data cleaning

In [1]:
# Install zipfile36 if you haven't already
# !pip install zipfile36

In [2]:
# Import our dependencies
import pandas as pd
import numpy as np
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

# SQL
from sqlalchemy import create_engine
import sqlite3 as sql

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import os
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Create dataframe
z = urlopen('https://archive.ics.uci.edu/ml/machine-learning-databases/00296/dataset_diabetes.zip')
myzip = ZipFile(BytesIO(z.read())).extract('dataset_diabetes/diabetic_data.csv')
df = pd.read_csv(myzip)
df.head(5)
target = ['A1Cresult']

In [ ]:
# Drop the non-beneficial ID columns, 'encounter_id' and 'patient_nbr'
df = df.drop(['patient_nbr'],1)

# Drop mostly empty columns, 'weight', 'payer_code', 'max_glu_serum', and 'medical_specialty'
df = df.drop(['weight', 'payer_code', 'max_glu_serum', 'medical_specialty', ],1)

# Replace '?' values to nulls
df.replace({'?': np.nan}, inplace=True)

# Replace 'None' values to nulls
df.replace({'None': np.nan}, inplace=True)

# Drop the null rows
df = df.dropna()

In [ ]:
# Convert the target column values to normal and high based on their values
x = {'Norm': 'normal'}   
df = df.replace(x)
x = dict.fromkeys(['>7', '>8'], 'high')    
df = df.replace(x)
df.reset_index(inplace=True, drop=True)

In [ ]:
# Determine the number of unique values in each column.
df.nunique()

In [ ]:
# Drop columns with only 1 value
df = df.drop(['acetohexamide', 'troglitazone', 'examide', 'citoglipton','glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone'],1)
df.nunique()

In [ ]:
# Drop variables with outliers 
df = df.drop(['number_inpatient', 'number_outpatient','number_emergency'],1)
df.nunique()

In [ ]:
# Info on the data frame data types, non_null, etc. 
df.info()

In [ ]:
# Looking at dataframe 
df.head()

In [ ]:
# Generate our categorical variable list
df_cat = df.dtypes[df.dtypes == "object"].index.tolist()
df_cat

In [ ]:
# Check the number of unique values in each column
df[df_cat].nunique()

In [ ]:
# Create SQLite engine using SQLAlechmey
engine = create_engine('sqlite:///diabetes_data.db', echo=False)
conn = engine.connect()
df.to_sql('diabetes', conn, index=False, if_exists='replace')

In [ ]:
# Check dataset
pd.read_sql('SELECT * FROM diabetes LIMIT 10', conn)

In [ ]:
# Create cleanup table
engine.execute('CREATE TABLE "cleaned_columns" ('
               'id BIGINT NOT NULL,'
               'diag_1c VARCHAR, '
               'diag_2c VARCHAR, '
               'diag_3c VARCHAR, '
               'PRIMARY KEY (id));')

In [ ]:
# Insert columns into new table to clean
engine.execute('INSERT INTO "cleaned_columns" '
               'SELECT encounter_id AS id, '
               'CASE '
                   'WHEN (diag_1 BETWEEN 390 AND 459) OR (diag_1 = 785) THEN "circulatory" '
                   'WHEN (diag_1 BETWEEN 460 AND 519) OR (diag_1 = 786) THEN "respiratory" '
                   'WHEN (diag_1 BETWEEN 520 AND 579) OR (diag_1 = 787) THEN "digestive" '
                   'WHEN (diag_1 BETWEEN 250 AND 251) THEN "diabetes" '
                   'WHEN (diag_1 BETWEEN 800 AND 999) THEN "injury" '
                   'WHEN (diag_1 BETWEEN 710 AND 739) THEN "musculoskeletal" '
                   'WHEN (diag_1 BETWEEN 580 AND 629) OR (diag_1 = 788) THEN "genitourinary" '
                   'WHEN (diag_1 BETWEEN 140 AND 239) THEN "neoplasms" '
                   'ELSE "other" '
               'END AS diag_1c, '
               'CASE '
                   'WHEN (diag_2 BETWEEN 390 AND 459) OR (diag_2 = 785) THEN "circulatory" '
                   'WHEN (diag_2 BETWEEN 460 AND 519) OR (diag_2 = 786) THEN "respiratory" '
                   'WHEN (diag_2 BETWEEN 520 AND 579) OR (diag_2 = 787) THEN "digestive" '
                   'WHEN (diag_2 BETWEEN 250 AND 251) THEN "diabetes" '
                   'WHEN (diag_2 BETWEEN 800 AND 999) THEN "injury" '
                   'WHEN (diag_2 BETWEEN 710 AND 739) THEN "musculoskeletal" '
                   'WHEN (diag_2 BETWEEN 580 AND 629) OR (diag_2 = 788) THEN "genitourinary" '
                   'WHEN (diag_2 BETWEEN 140 AND 239) THEN "neoplasms" '
                   'ELSE "other" '
               'END AS diag_2c, '
               'CASE '
                   'WHEN (diag_3 BETWEEN 390 AND 459) OR (diag_3 = 785) THEN "circulatory" '
                   'WHEN (diag_3 BETWEEN 460 AND 519) OR (diag_3 = 786) THEN "respiratory" '
                   'WHEN (diag_3 BETWEEN 520 AND 579) OR (diag_3 = 787) THEN "digestive" '
                   'WHEN (diag_3 BETWEEN 250 AND 251) THEN "diabetes" '
                   'WHEN (diag_3 BETWEEN 800 AND 999) THEN "injury" '
                   'WHEN (diag_3 BETWEEN 710 AND 739) THEN "musculoskeletal" '
                   'WHEN (diag_3 BETWEEN 580 AND 629) OR (diag_3 = 788) THEN "genitourinary" '
                   'WHEN (diag_3 BETWEEN 140 AND 239) THEN "neoplasms" '
                   'ELSE "other" '
               'END AS diag_3c '
               'FROM diabetes;')

In [ ]:
# Check data
pd.read_sql('SELECT * FROM cleaned_columns LIMIT 10', conn)

In [ ]:
# Join diabetes and cleaned_columns tables
diabetes_df = pd.read_sql('SELECT * FROM diabetes JOIN cleaned_columns ON diabetes.encounter_id = cleaned_columns.id', conn)
diabetes_df 

In [ ]:
# Check Datatypes
diabetes_df.dtypes

In [ ]:
# View DataFrame
diabetes_df

In [ ]:
# Generate our categorical variable list
diabetes_df_cat = diabetes_df.dtypes[diabetes_df.dtypes == "object"].index.tolist()
diabetes_df_cat

In [ ]:
# Drop the non-beneficial ID and diag columns, 'encounter_id', 'id', 'diag_1', 'diag_2', and 'diag_3'
diabetes_2_df = diabetes_df.drop(['encounter_id', 'id', 'diag_1', 'diag_2', 'diag_3'],1)

In [ ]:
diabetes_2_df 

In [ ]:
# Check data
diabetes_2_df.dtypes

In [ ]:
# Generate our categorical variable list
diabetes_2_df_cat = diabetes_2_df.dtypes[diabetes_2_df.dtypes == "object"].index.tolist()
diabetes_2_df_cat

In [ ]:
# OneHot encode the categorical data
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(diabetes_2_df[diabetes_2_df_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names_out(diabetes_2_df_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
diabetes_2_df = diabetes_2_df.merge(encode_df,left_index=True, right_index=True)
diabetes_2_df = diabetes_2_df.drop(diabetes_2_df_cat,1)
diabetes_2_df.head()

In [ ]:
# Remove target from features data
y = diabetes_2_df['A1Cresult_high']
X = diabetes_2_df.drop(columns=["A1Cresult_high","A1Cresult_normal"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

In [ ]:
# Create new dataframe with top only 8 features
top_8_df = diabetes_2_df[['num_lab_procedures','num_medications','time_in_hospital','num_procedures','number_diagnoses','discharge_disposition_id','admission_type_id','admission_source_id']] 
top_8_df.head(20)

In [ ]:
# Remove target from features data
y = diabetes_2_df['A1Cresult_high']
X = top_8_df

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

In [ ]:
# Trying PCA on original data with 95% variance
pca = PCA(n_components = 0.95)
pca.fit(diabetes_2_df)
reduced = pca.transform(diabetes_2_df)

In [ ]:
# Call into dataframe 
transform_pca = pd.DataFrame(reduced)
transform_pca

In [ ]:
# Trying PCA on original data with 99% variance
pca = PCA(n_components = 0.99)
pca.fit(diabetes_2_df)
reduced_2 = pca.transform(diabetes_2_df)

In [ ]:
# Call into dataframe 
pd.DataFrame(reduced_2)

In [ ]:
# Running Random Forest with the top 5 components from PCA
# Remove target from features data
y = diabetes_2_df['A1Cresult_high']
X = transform_pca
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = len(X_train_scaled[0])*3
hidden_nodes_layer2 = len(X_train_scaled[0])*2
hidden_nodes_layer3 = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save("ML_Diabetes.h5")